# Check funding rate entities

* Query positions to ensure `netFunding` is recorded properly
* Check `fundingRates` value on the entities for historical funding rate chart


In [49]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import sqlite3
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px

load_dotenv()

pio.renderers.default = 'notebook'

In [50]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


## Historical funding rate

In [57]:
df_position = pd.read_sql_query('SELECT * FROM positions', con)
df_funding = pd.read_sql_query('SELECT * FROM funding_rates ORDER BY asset, timestamp', con)


In [58]:
fig = px.line(
    df_funding,
    x='timestamp',
    y='fundingRate',
    color='asset'
)

fig.show()

## Look at a specific market

In [59]:
asset = 'SOL'
last_price = df_position[(df_position['asset'] == asset)].sort_values('timestamp', ascending=False).iloc[0, :]['lastPrice']

df_position['sizeUsd'] = df_position.apply(lambda x: x['size'] * last_price, axis=1)
df_long = df_position.loc[(df_position['asset'] == asset) & (df_position['isOpen'] == True) & (df_position['size'] > 0), :].sort_values('size', ascending=False)
df_short = df_position.loc[(df_position['asset'] == asset) & (df_position['isOpen'] == True) & (df_position['size'] < 0), :].sort_values('size', ascending=True)

long_oi = df_long['size'].sum()
short_oi = abs(df_short['size'].sum())
total_oi = long_oi + short_oi

df_long['oiSide'] = df_long['size'].apply(lambda x: x / long_oi)
df_short['oiSide'] = df_short['size'].apply(lambda x: -1 * x / short_oi)
df_long['oiTotal'] = df_long['size'].apply(lambda x: x / total_oi)
df_short['oiTotal'] = df_short['size'].apply(lambda x: -1 * x / total_oi)

display_cols = [
    'account',
    'asset',
    'margin',
    'size',
    'sizeUsd',
    'oiSide',
    'oiTotal',
    'avgEntryPrice',
    'pnl',
    'totalVolume'
]

print(f'Long  OI: {round(df_long.sizeUsd.sum(), 2)}')
print(f'Short OI: {round(-df_short.sizeUsd.sum(), 2)}')


Long  OI: 2007531.1
Short OI: 2013733.32


In [60]:
df_long[display_cols]

,account,asset,margin,size,sizeUsd,oiSide,oiTotal,avgEntryPrice,pnl,totalVolume
1815,0xe8c19db00287e3536075114b2576c70773e039bd,SOL,306385.350724,88849.400267,2.001365e+06,0.996928,0.497695,22.583447,0.0,2.006526e+06
1779,0xf7b18e107eb36797f4ce36de756630b9c30969ad,SOL,393.524574,112.265100,2.528812e+03,0.001260,0.000629,22.345323,0.0,2.508600e+03
1863,0x75c319a263b52db1f499a4abb4b9900b97bd1668,SOL,47.484750,57.033200,1.284693e+03,0.000640,0.000319,22.353000,0.0,1.274863e+03
1853,0x2241d1c14efe86d88b537ecbf4c3187a58771592,SOL,61.843883,56.984300,1.283592e+03,0.000639,0.000319,23.228914,0.0,1.323683e+03
1828,0xac9f5a0a1c0f6862f2485b2f1390d31bfc90f4ab,SOL,47.157242,30.453200,6.859693e+02,0.000342,0.000171,23.061560,0.0,7.022983e+02
1809,0x6fb737d1ebb73cda6cfa36fd16d9273065d1b084,SOL,190.821128,10.050483,2.263908e+02,0.000113,0.000056,22.410333,0.0,2.252347e+02
1799,0x40dfc27ca78b6fbcfb78e91d3d2426cf6a281e39,SOL,47.880112,6.960708,1.567925e+02,0.000078,0.000039,21.529485,0.0,1.498605e+02


In [61]:
df_short[display_cols]


,account,asset,margin,size,sizeUsd,oiSide,oiTotal,avgEntryPrice,pnl,totalVolume
1837,0xceec48581b3145a575508719f45da07dc57fa7ce,SOL,99988.164889,-21000.000000,-473032.588138,0.234903,0.117633,23.450306,2833.631363,1.054138e+06
1800,0x01c8c06eb7a35f978f5629461552f57bc9783acd,SOL,47102.708269,-10325.000000,-232574.355835,0.115494,0.057836,22.694233,1068.261344,2.785790e+05
1812,0x323f0aa372a0467fd430d844da4edb0603865c42,SOL,31322.992550,-10018.790121,-225676.867660,0.112069,0.056121,23.065205,0.000000,2.310854e+05
1849,0x80c27e00a00d80a4f83ff9c2dfdd562e95dc9695,SOL,50689.501637,-8586.297393,-193409.451343,0.096045,0.048097,23.314910,0.000000,2.001887e+05
1818,0xbf1cfce65043191daa5f701398bc6d3112d47e29,SOL,76328.334087,-7500.000000,-168940.210049,0.083894,0.042012,22.801642,0.000000,1.710123e+05
1834,0x8f22cee99b55ffba8c55f27181d14bcd34e202ed,SOL,19162.076974,-7000.000000,-157677.529379,0.078301,0.039211,23.380167,0.000000,1.636612e+05
1819,0x0514f2f3e0277c47117e3f33d9390efb0acfbdfe,SOL,50814.361197,-5555.000000,-125128.382243,0.062138,0.031117,22.718355,0.000000,1.262005e+05
1839,0x2cd3a242d493bb8ae3369d3ee2b83b5bf4fe82d7,SOL,9932.423603,-3370.000000,-75910.467716,0.037696,0.018877,23.674325,0.000000,7.978247e+04
1848,0x2ea1fd954b5d21ff704e37dfe50f796eec4620cd,SOL,23821.821607,-2566.959589,-57821.692288,0.028714,0.014379,23.398957,0.000000,6.006418e+04
1851,0x9faf4af74102d5a5b5e74f5c15b8d74bf70bc0ab,SOL,16431.088807,-2500.000000,-56313.403350,0.027965,0.014004,23.160687,0.000000,5.790172e+04
